In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import scanpy as sc
import os as os
from scipy.stats import spearmanr

2021-09-14 16:08:16.988402: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Load models and selected genes

In [2]:
array_of_models = []
for i in range(7):
    array_of_models.append(tf.keras.models.load_model('Best_performance_model_for_clsuter_'+str(i)+'_final.h5'))

2021-09-14 16:09:03.652987: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-09-14 16:09:03.718203: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.837GHz coreCount: 28 deviceMemorySize: 11.75GiB deviceMemoryBandwidth: 335.32GiB/s
2021-09-14 16:09:03.718299: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-09-14 16:09:03.724422: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-09-14 16:09:03.724572: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-09-14 16:09:03.726605: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcu

In [3]:
selected_gene_index = []
for i in range(7):
    selected_gene_index.append(np.load('Selected_genes/selected_gene'+str(i)+'.npy'))

In [6]:
gene_names = np.load('Selected_genes/Gene_names.npy', allow_pickle=True)

# Preprocessing and prediction

In [10]:
def Preprocessing(countmatrix,colname_of_gene_name, colname_of_counts, gene_names):
    X = []
    for i in gene_names:
        if i in countmatrix[colname_of_gene_name]:
            X.append(countmatrix[countmatrix[colname_of_gene_name]==i][colname_of_counts])
        else:
            print('Warning! Missing an expression!')
            X.append(0)
    return X

In [11]:
def PredictWithSelection(model_list, X_test, selected_gene_index):
    y_prediction_list = []
    for cluster in range(0,y_test[0].shape[0]):
        print('Deconvoluting cluster '+str(cluster))
        sample_id = 0
        y_prediction =[]
        if cluster == y_test[0].shape[0]-1:
            for X in X_test[:,selected_gene_index[cluster]]:
                print('Deconvoluting sample '+str(sample_id))
                y = model_list[cluster].predict(np.array([X]))[0]
                y_prediction.append(y.flatten()[0]/10)
                sample_id+=1
        else:
            for X in X_test[:,selected_gene_index[cluster]]:
                print('Deconvoluting sample '+str(sample_id))
                y = model_list[cluster].predict(np.array([X]))[0]
                y_prediction.append(y.flatten()[0])
                sample_id+=1
        y_prediction = np.array(y_prediction)
        y_prediction_list.append(y_prediction)
    return(y_prediction_list)

In [12]:
'''
1. run Preprocessing function (this function processes one sample at a time, 
process batches through the means of loops)
2. run PredictWithSelection function to get a list with predicted cell proportions
'''

'\n1. run Preprocessing function (this function processes one sample at a time, \nprocess batches through the means of loops)\n2. run PredictWithSelection function to get a list with predicted cell proportions\n'